<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - User Story 1

### Access of pre-computed ARD data for Sentinel 1 and 2 over a large area.

1. Large Area from VITO. Pre computed 10-20 tiles. 
Overlay Sentinel-2 tiles
2. Show that the grids are aligned. Overlay the data as geotiff to show the alignment.
3. Show/visualize metadata -> ask Jeroen how to get it

DEV MEETING QUESTION:
1. What happens if the bbox covers more than one tile? Different timesteps?
2. How to use a batch job for a large area?

In [3]:
# Import necessary libraries
import time
import rasterio
import numpy as np
import xarray as xr
import openeo
from ipyleaflet import *
from ipywidgets import *
from traitlets import link
import shapely.geometry
from bqplot import Lines, Figure, LinearScale, DateScale, Axis
from datetime import datetime

## Run the next cell and select the Area Of Interest using the rectangle button.

In [4]:
label = Label(value='[*] Loading data from OpenEO...')
def openeo_timeseries(feature_collection,band_name='B04',init=False):
    global label
    if not init:
        display(label)
    else:
        label.value = '[*] Loading data from OpenEO...'
    connection = openeo.connect("https://openeo-dev.vito.be").authenticate_basic('guest','guest123')

    coords = feature_collection['features'][0]['geometry']['coordinates'][0]
    polygon = shapely.geometry.Polygon(coords)
    bbox = polygon.bounds

    cloudmask = connection.load_collection("SENTINEL2_L2A_SENTINELHUB",
                                     spatial_extent={'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':'EPSG:4326'},
                                     temporal_extent=["2018-02-01", "2020-02-01"],
                                     bands=['CLM'])

    l1c = connection.load_collection("SENTINEL2_L1C_SENTINELHUB",
            spatial_extent={'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':'EPSG:4326'},
                                     temporal_extent=["2018-02-01", "2020-02-01"],
                                     bands=[band_name])

    l2a = l1c.mask(cloudmask).filter_bands(band_name)
    l2a.download("./openeo-rgb-l2a_area.nc", format="NETCDF")
    return

center = [46.88, 10.45]
zoom = 7
m = Map(center=center, zoom=zoom)
feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

draw = DrawControl(
    circlemarker={}, polyline={}, polygon={}, marker = {},
    
    rectangle = {"shapeOptions": {
               "original": {},  # work around for bug in Leaflet.Draw
               "editing": {},   # work around for bug in Leaflet.Draw
    }})

# geojson layer with hover handler
with open('./Shapefiles/Europe.geojson') as f:
    data = json.load(f)
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': None,
        'fillOpacity': 0.1    }
s2_tiles_layer = GeoJSON(data=data)
m.add_layer(s2_tiles_layer)

m.add_control(draw)
init = False
widget_control = None
def handle_draw(target, action, geo_json):
    global init, widget_control
    if init:
        m.remove_control(widget_control)
    feature_collection['features'] = []
    feature_collection['features'].append(geo_json)
    figure = openeo_timeseries(feature_collection,band_name,init)
    widget_control = WidgetControl(widget=figure, position='bottomright')
    m.add_control(widget_control)
    if not init:
        m.add_control(basemap_control)
    init = True

# Adds a Dropdown widget
bands = ['B08','B04','B02']
band_name = 'B04'
dropdown = Dropdown(
    options=bands,
    value='B04',
    description='Bands'
)

# Handles Dropdown control event
def on_click(change):
    global band_name
    band_name = change['new']
    global init, widget_control
    if init:
        m.remove_control(widget_control)
    figure = openeo_timeseries(feature_collection,band_name,init)
    widget_control = WidgetControl(widget=figure, position='bottomright')
    m.add_control(widget_control)
    init = True    
dropdown.observe(on_click, 'value')

# Adds control to the map
basemap_control = WidgetControl(widget=dropdown, position='topright')
draw.on_draw(handle_draw)

display(m)

Map(center=[46.88, 10.45], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

Label(value='[*] Loading data from OpenEO...')